In [1]:
import getpass
import os

# Set LangSmith Environment Variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith")


In [2]:
# Set up google llm model for RAG usage
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


print("Initializing chat model!")
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
print("Chat model initialization over!")

Initializing chat model!
Chat model initialization over!


In [3]:
print("Initializing embedder model!")
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("Embedder model initialized!")

Initializing embedder model!


/opt/miniconda3/envs/rag-agent-musings/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedder model initialized!


In [4]:
print("Initialize vector store!")
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print("Vector store initialized!")

Initialize vector store!
Vector store initialized!


In [5]:
# Loading documents for RAG use

# get beautiful soup
import bs4
# get Document Loaders
from langchain_community.document_loaders import WebBaseLoader
desired_webpage_link = "https://lilianweng.github.io/posts/2023-06-23-agent/"
# the original tutorial only wanted us to keep html tags that
# contain any of the class names listed in the tuple assigned to the 
# class_ keyword, in the below variable. We discard all other html tags.
bs4_filtered = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

webpage_loader = WebBaseLoader(
  web_paths=(desired_webpage_link,),
  bs_kwargs={"parse_only": bs4_filtered}
)

documents = webpage_loader.load()
#for key in documents[0]:
  #print(f"New key: {key}")
#print(f"how many characters are in our page: {len(documents[0].page_content)}")
#print(f"The first two thousand characters of the document: {documents[0].page_content[:2000]}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create a text splitter instance
recursive_text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
  add_start_index=True,
)

# obtain splitted text. our document is now a bunch of document shards.
splitted_text = recursive_text_splitter.split_documents(documents)
#print(f"The splits we got back:\n\n{splitted_text[4].page_content}")
#print(f"Number of splits:{len(splitted_text)}")

# We've loaded in our document. 
Split it up into text chunks, to allow our model to access bits of the data that fit into the LLM context window.

Now, it's time to put these chunks into a storage format the LLM can access.

In [7]:
# the vector store was previously initialized with our defined embeddings
# in one command sequence, we tell the vector store to embed each of our document chunks.
# we then store the embedded versions of these chunks into our vector store, for 
# later retrieval
vector_store__document_embedded_vectors = vector_store.add_documents(splitted_text)


In [8]:
print(f"vector store shape: {vector_store__document_embedded_vectors}")

vector store shape: ['388cc8d7-6750-46e6-838d-8d9f43b34cb0', 'c09a6f2a-b6bb-443e-9210-02c5d0f68f67', '54796ac4-90ed-4af2-9720-94112d81533d', '6f87c8b5-46c2-4003-88ab-57a885e6503a', '2a30e582-f689-454a-9610-42b3d69fc35b', 'd44e2189-30a7-4066-a5fe-0e4dcdd1380a', 'd24b4562-c796-4ccd-b628-dbc1538e7607', '74c83b49-80a9-4599-86f4-94142db15c31', '3ea4fb0e-7775-42ae-b58b-766c1c1eea5c', '4b58cbc3-e2ea-4503-b6ea-adfa45d96ac0', '4762e501-0d4c-44e8-909d-671401b87163', 'cb4753e4-deae-42b8-a31b-4ebf70e366a2', 'ce413bac-18fe-4e64-b622-4961c29f0630', '6d5b98b1-ac3a-40f5-bbaf-1bf0399ed0e2', '315014e9-9bc8-4fd7-b226-130fe17afdbc', 'd8789dcb-2915-4682-94db-ee88117d5b7b', '72d3d277-1f8f-48b5-9a17-7b7af49c1fee', 'a133c656-6f76-49b6-8579-6b7466a12a31', 'f6d710d9-1951-4fca-a98f-0fd13577d45a', '60654684-4dda-4b9a-8535-66ba22d7fa22', 'bde17f43-ddfc-4ff0-81b2-a98c24d2db16', '0c722aba-95cb-4420-b6f1-07e2b2cad575', 'b8b5795e-4256-4d9b-ab7c-8a836d75184f', 'a19d9c58-ba33-416c-a7ec-3db27d2ee135', '86efc2ed-2e4e-4b8d

# We now should be able to answer User Queries
By accessing our vector store and returning the relevant chunks of text, from Lillian's blog post, that answers the user's questions.

## Onto Retrieval And Generation

In [9]:
# Get RAG prompt
from langchain import hub

# The actual RAG prompt
rag_prompt = hub.pull("rlm/rag-prompt")
#print(f"Actual template response: {rag_prompt.messages[0].prompt.template}.")

example_rag_message = rag_prompt.invoke({
  "context": "context here",
  "question": "question here"
}).to_messages() # Example of passing in context and question to a full RAG prompt.

print(f"Example rag message: {example_rag_message}")

Example rag message: [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: question here \nContext: context here \nAnswer:", additional_kwargs={}, response_metadata={})]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
## Use LangGraph to generate for our RAG application.

# Begin the state definition phase, where we discuss what type of data we want our RAG model to process.

from langchain_core.documents import Document
from typing_extensions import List, TypedDict

# Create the actual state, to define what data our RAG model handles

class RAG_State(TypedDict):
  question: str
  context: List[Document] # remember, context should be several chunks of a document
  answer: str

def retrieve(rag_state: RAG_State):
  """
  Retrieves relevant data, based on our user question, from the vector store

  Args:
    rag_state: A dictionary which, for this problem, contains the user question, in the key 'question'. Has to be of type RAG_State
  
  Returns:
    All relevant context document chunks. Should be a dictionary of the following form: {str ; List[Document]}
  """
  similar_document_chunks = vector_store.similarity_search(rag_state["question"])
  return {"context" : similar_document_chunks}

def generate(rag_state: RAG_State):
  """
  Generate appropriate RAG response to the user query.

  Args:
    rag_state: A dictionary which, for this problem, contains the user question, in the key 'question', and the context of the question, in the key 'context'. Has to be of type RAG_State
  
  Returns:
    The model's response. Should be a dictionary of the following form: {str ; ?}
  """
  documents_contents = "\n\n".join(doc.page_content for doc in rag_state["context"])
  formalized_RAG_prompt = rag_prompt.invoke({"question": rag_state["question"], "context": rag_state["context"]})
  llm_RAG_response = llm.invoke(formalized_RAG_prompt)
  return {"answer" : llm_RAG_response.content}

In [ ]:
# Create a graph object, to link together the retrieval and generate steps.
# Note that our retrieval and generate steps are simple as of now, but can be
# made more complex.

from langgraph.graph import START, StateGraph # START is a special Start Node

rag_agent_maker = StateGraph(RAG_State).add_sequence([retrieve, generate])
rag_agent_maker.add_edge(START, "retrieve") # links the special Start node to the retrieve node sequence we defined above.
rag_agent = rag_agent_maker.compile() # now we have made our RAG agent.